# Анализ базы данных сервиса для чтения книг по подписке

## Цель исследования

Цель — проанализировать базу данных с информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

## Описание данных

Таблица **`books`**
Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

Таблица **`authors`**
Содержит данные об авторах:
- `author_id` — идентификатор автора;
- `author` — имя автора.

Таблица **`publishers`**
Содержит данные об издательствах:
- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

Таблица **`ratings`**
Содержит данные о пользовательских оценках книг:
- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

Таблица **`reviews`**
Содержит данные о пользовательских обзорах:
- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя автора обзора;
- `text` — текст обзора.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Вывод первых 5 строк каждой таблицы

In [2]:
# Первые 5 строк таблицы books
print()
print('Первые 5 строк таблицы books')
books_limit_5 = ''' SELECT *
                    FROM books
                    LIMIT 5
                '''

con=engine.connect()

pd.io.sql.read_sql(sql=text(books_limit_5), con = con)


Первые 5 строк таблицы books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
# Первые 5 строк таблицы authors
print()
print('Первые 5 строк таблицы authors')
authors_limit_5 = '''SELECT *
                     FROM authors
                     LIMIT 5
                  '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(authors_limit_5), con = con)


Первые 5 строк таблицы authors


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [4]:
# Первые 5 строк таблицы ratings
print()
print('Первые 5 строк таблицы ratings')
ratings_limit_5 = '''SELECT *
                     FROM ratings
                     LIMIT 5
                  '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(ratings_limit_5), con = con)


Первые 5 строк таблицы ratings


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [5]:
# Первые 5 строк таблицы reviews
print()
print('Первые 5 строк таблицы reviews')
reviews_limit_5 = '''SELECT *
                     FROM reviews
                     LIMIT 5
                  '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(reviews_limit_5), con = con)


Первые 5 строк таблицы reviews


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [6]:
# Первые 5 строк таблицы publishers
print()
print('Первые 5 строк таблицы publishers')
publishers_limit_5 = '''SELECT *
                        FROM publishers
                        LIMIT 5
                     '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(publishers_limit_5), con = con)


Первые 5 строк таблицы publishers


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


## Задание 1 - Посчитайте, сколько книг вышло после 1 января 2000 года

In [7]:
books_cnt = '''SELECT COUNT(book_id)
               FROM books
               WHERE publication_date > '2000-01-01'
            '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(books_cnt), con = con)

,count
0,819


После 1 января 2000 года вышло 819 книг

## Задание 2 - Для каждой книги посчитайте количество обзоров и среднюю оценку

In [8]:
books_rev_cnt_rat_avg = '''SELECT b.book_id,
                                 b.title,
                                COUNT(DISTINCT rev.review_id) AS reviews_cnt,
                                ROUND(AVG(rat.rating),2) AS rating_avg
                           FROM books AS b
                           LEFT JOIN ratings AS rat ON b.book_id = rat.book_id
                           LEFT JOIN reviews AS rev ON b.book_id = rev.book_id
                           GROUP BY b.book_id
            '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(books_rev_cnt_rat_avg), con = con)

,book_id,title,reviews_cnt,rating_avg
0,1,'Salem's Lot,2,3.67
1,2,1 000 Places to See Before You Die,1,2.50
2,3,13 Little Blue Envelopes (Little Blue Envelope...,3,4.67
3,4,1491: New Revelations of the Americas Before C...,2,4.50
4,5,1776,4,4.00
...,...,...,...,...
995,996,Wyrd Sisters (Discworld #6; Witches #2),3,3.67
996,997,Xenocide (Ender's Saga #3),3,3.40
997,998,Year of Wonders,4,3.20
998,999,You Suck (A Love Story #2),2,4.50


Количество обзоров и средний рейтинг для каждой книги посчитан

## Задание 3 - Определите издательство, которое выпустило наибольшее число книг толще 50 страниц - так вы исключите из анализа брошюры

In [9]:
publisher_top_1 = '''WITH
                     publishers_top_1 AS (SELECT p.publisher,
                                          COUNT(b.book_id) as books_cnt
                                          FROM publishers AS p
                                          LEFT JOIN books AS b ON p.publisher_id=b.publisher_id
                                          WHERE b.num_pages > 50
                                          GROUP BY p.publisher
                                          ORDER BY books_cnt DESC
                                          LIMIT 1
                                          )
                    SELECT publisher
                    FROM publishers_top_1
                        '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(publisher_top_1), con = con)

,publisher
0,Penguin Books


Издательство Penguin Books выпустило больше всех книг толще 50 страниц

## Задание 4 - Определите автора с самой высокой средней оценкой книг - учитывайте только книги с 50 и более оценками

In [10]:
author_top_1 = '''WITH
                  book_rating AS (SELECT b.book_id,
                                             b.author_id,
                                             AVG(r.rating) AS rating_avg,
                                             COUNT(r.rating_id) AS rating_cnt
                                      FROM books AS b
                                      LEFT JOIN ratings AS r ON b.book_id = r.book_id
                                      GROUP BY b.book_id
                                      HAVING COUNT(r.rating_id) >=50
                                      ORDER BY rating_avg DESC
                                      ),
                                      
                   author_rating_avg AS (SELECT author_id,
                                               AVG(rating_avg) AS author_rating_avg
                                        FROM book_rating
                                        GROUP BY author_id
                                        ORDER BY author_rating_avg DESC
                                        LIMIT 1)
                  SELECT a.author
                  FROM authors AS a
                  JOIN author_rating_avg AS ar ON a.author_id = ar.author_id

                '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(author_top_1), con = con)

,author
0,J.K. Rowling/Mary GrandPré


Авторами с самой высокой средней оценкой книг являются J.K. Rowling/Mary GrandPré

## Задание 5 - Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [11]:
user_review_avg = '''WITH
                     cnt_rev_by_user AS (SELECT COUNT(reviews.review_id) AS reviews_cnt
                                         FROM reviews
                                         WHERE username IN (SELECT username
                                                            FROM ratings
                                                            GROUP BY username
                                                            HAVING COUNT(rating_id) > 48)
                                         GROUP BY username)
             
                     SELECT AVG(reviews_cnt) AS reviews_avg
                     FROM cnt_rev_by_user
                     
                 '''
                      
                      
                    
con=engine.connect()
pd.io.sql.read_sql(sql=text(user_review_avg), con = con)

,reviews_avg
0,24.0


Среднее количество обзоров от пользователей, которые поставили больше 48 оценок - 24

## Вывод

После решения поставленных садач можно сделать вывод:

- После 1 января 2000 года вышло 819 книг
- Посчитали количество обзоров и средний рейтинг для каждой книги
- Издательство Penguin Books выпустило больше всех книг толще 50 страниц
- Авторами с самой высокой средней оценкой книг являются J.K. Rowling/Mary GrandPré
- Среднее количество обзоров от пользователей, которые поставили больше 48 оценок - 24